Spark homework

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

Start Spark session

In [2]:
spark = SparkSession.builder \
    .master('local') \
    .appName('Spark-Homework') \
    .getOrCreate()
spark

21/08/25 13:42:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# Task 1 
```
Upload generated train.parquet , destinations.csv, struct_cities.json files from previous module Homework to Jupyter workspace. Create dataframes  and tempViews upon the files.
```

In [3]:
HDFS_HOST = '172.17.0.3'
WAREHOUSE_PATH = '/user/hive/warehouse'


paths =  {'destinations' : 'hdfs://172.17.0.2/user/hive/warehouse/destinations/destinations.csv',
        'struct_cities': 'hdfs://172.17.0.2/user/hive/warehouse/struct_cities/struct_cities.json',
        'train': 'hdfs://172.17.0.2/user/hive/warehouse/staging/train.parquet/'}


## Read parquet file

In [4]:
train_df = spark.read.parquet(f'hdfs://{HDFS_HOST}/{WAREHOUSE_PATH}/staging/train.parquet',
                        header = 'true',
                        inferSchema = True
                                )
train_df.printSchema()


root
 |-- date_time: string (nullable = true)
 |-- site_name: integer (nullable = true)
 |-- posa_continent: integer (nullable = true)
 |-- user_location_country: integer (nullable = true)
 |-- user_location_region: integer (nullable = true)
 |-- user_location_city: integer (nullable = true)
 |-- orig_destination_distance: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- is_mobile: integer (nullable = true)
 |-- is_package: integer (nullable = true)
 |-- channel: integer (nullable = true)
 |-- srch_ci: string (nullable = true)
 |-- srch_co: string (nullable = true)
 |-- srch_adults_cnt: integer (nullable = true)
 |-- srch_children_cnt: integer (nullable = true)
 |-- srch_rm_cnt: integer (nullable = true)
 |-- srch_destination_id: integer (nullable = true)
 |-- srch_destination_type_id: integer (nullable = true)
 |-- is_booking: integer (nullable = true)
 |-- cnt: integer (nullable = true)
 |-- hotel_continent: integer (nullable = true)
 |-- hotel_country: integer (

## Read json

In [5]:
struct_cities_df = spark.read.json(f'hdfs://{HDFS_HOST}/{WAREHOUSE_PATH}/struct_cities/struct_cities.json')
struct_cities_df.printSchema()

In [6]:
struct_cities_df.show(3, truncate = False, vertical = True)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------
 cities       | [{FALSE, FALSE, FALSE, Santa Cruz BalanyГЎ, 0, America/Guatemala}, {FALSE, TRUE, TRUE, San Pedro SacatepГ©quez, 0, America/Guatemala}]                         
 country_code | 0                                                                                                                                                              
-RECORD 1----------------------------------------------------------------------------------------------------------------------------------------------------------------------
 cities       | [{FALSE, FALSE, TRUE, AtyrГЎ, 1, America/Asuncion}, {FALSE, FALSE, FALSE, Antequera, 1, America/Asuncion}]                                                     
 country_code | 1                                                                                                       

## Read csv

In [11]:
destinations_df = spark.read.csv(f'hdfs://{HDFS_HOST}/{WAREHOUSE_PATH}/destinations/destinations.csv',
                        header = 'true',
                        inferSchema = True
                                )
destinations_df.printSchema()

root
 |-- srch_destination_id: integer (nullable = true)
 |-- d1: double (nullable = true)
 |-- d2: double (nullable = true)
 |-- d3: double (nullable = true)
 |-- d4: double (nullable = true)
 |-- d5: double (nullable = true)
 |-- d6: double (nullable = true)
 |-- d7: double (nullable = true)
 |-- d8: double (nullable = true)
 |-- d9: double (nullable = true)
 |-- d10: double (nullable = true)
 |-- d11: double (nullable = true)
 |-- d12: double (nullable = true)
 |-- d13: double (nullable = true)
 |-- d14: double (nullable = true)
 |-- d15: double (nullable = true)
 |-- d16: double (nullable = true)
 |-- d17: double (nullable = true)
 |-- d18: double (nullable = true)
 |-- d19: double (nullable = true)
 |-- d20: double (nullable = true)
 |-- d21: double (nullable = true)
 |-- d22: double (nullable = true)
 |-- d23: double (nullable = true)
 |-- d24: double (nullable = true)
 |-- d25: double (nullable = true)
 |-- d26: double (nullable = true)
 |-- d27: double (nullable = true)
 |-- d2

## Create temp Views

In [14]:
train_df.createOrReplaceTempView('train')
struct_cities_df.createOrReplaceTempView('struct_cities')
destinations_df.createOrReplaceTempView('destinations')

## Task 2
```
Please complete the following tasks in 2 ways: 
•	using dataframe tansformations 
•	using temporary views and SQL
2.1 Find top 3 most popular hotels between couples. (treat hotel as composite key of continent, country and market). Save results in dataframe and write to task2_1.csv file. 
2.2  Find the most popular country where hotels are booked and searched from the same country. Save results in dataframe and write to task2_2.parquet file with overwrite mode.  
2.3  Find top 3 hotels where people with children are interested but not booked in the end.  Save as tempView.

```

In [34]:
# Functions for processing
from pyspark.sql import functions as F

### 2.1 Find top 3 most popular hotels between couples. (treat hotel as composite key of continent, country and market). Save results in dataframe and write to task2_1.csv file. 

#### Using SQL

In [42]:
spark.sql('''
        select hotel_continent, hotel_country, hotel_market, count(*) as count 
        from train 
        where srch_adults_cnt = 2 
        group by hotel_continent, hotel_country, hotel_market  
        order by count(*) desc 
        limit 3
        ''') \
        .show()

+---------------+-------------+------------+-------+
|hotel_continent|hotel_country|hotel_market|  count|
+---------------+-------------+------------+-------+
|              2|           50|         628|1190143|
|              2|           50|         675|1007502|
|              4|            8|         110| 588213|
+---------------+-------------+------------+-------+



#### Using DF transformations

In [84]:
top3_hotel = train_df.select('hotel_continent', 'hotel_country', 'hotel_market') \
            .filter(train_df.srch_adults_cnt == 2) \
            .groupBy('hotel_continent', 'hotel_country', 'hotel_market').count() \
            .orderBy('count', ascending = False) \
            .limit(3) 
top3_hotel.show()



+---------------+-------------+------------+-------+
|hotel_continent|hotel_country|hotel_market|  count|
+---------------+-------------+------------+-------+
|              2|           50|         628|1190143|
|              2|           50|         675|1007502|
|              4|            8|         110| 588213|
+---------------+-------------+------------+-------+



### Write to single file

In [91]:
top3_hotel.coalesce(1).write.csv('task2_1.csv', mode = 'overwrite')